# Kaggle project

In [15]:
def write_to_file(filename, X, y):
    with open(filename, 'w') as train_file:
        for (doc, label) in zip(X, y):
            train_file.write('__label__{0} {1}\n'.format(label, doc))

In [16]:
# importing files and preprocessing
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing

names1 = ['text', 'sen']  
data = pd.read_csv("products_sentiment_train.tsv", sep = '\t', names = names1)
test = pd.read_csv("products_sentiment_test.tsv", sep = '\t')
print('Entries: ', len(data))
data['text'] = [' '.join([elem for elem in re.sub(r'\w*\d\w*', '', elem).split(' ') ]) for elem in data['text']]
data = data.dropna(subset=['text'])
X_data, y_data = data['text'], data['sen']
X_test = test['text']
print('Entries: ', len(data))
le = preprocessing.LabelEncoder()
le.fit(y_data)
y_encoded = le.transform(y_data)

write_to_file('test.txt', X_test, np.ones(len(X_test)))
write_to_file('train.txt', X_data, y_data)

Entries:  2000
Entries:  2000


In [17]:
test.head()

Id                                               text
0   0  so , why the small digital elph , rather than ...
1   1  3/4 way through the first disk we played on it...
2   2  better for the zen micro is outlook compatibil...
3   3    6 . play gameboy color games on it with goboy .
4   4  likewise , i 've heard norton 2004 professiona...

# Attamp to make own word model with existing dataset

In [ ]:
def write_train_test(filename, X):
    with open(filename, 'w') as train_file:
        for descr in X:
            train_file.write('{}\n'.format(descr))

write_train_test('train+test.txt', np.array(pd.concat([X_data,X_test], axis =1)))

In [ ]:
%%time
!fasttext skipgram -input train+test.txt -output model_skipgram -dim 300 -loss hs -ws 5 -epoch 4 -minCount 1

# Try preprocessing

In [ ]:
!cat train.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > train.preprocessed.txt
!cat test.txt | sed -e "s/\([.\!?,'/()]\)/ \1 /g" | tr "[:upper:]" "[:lower:]" > test.preprocessed.txt

# Making predictions and submiting results

In [20]:
# try fasttext facebook lib with pretrained word vector
import fasttext
model_final = fasttext.supervised('train.preprocessed.txt', 'model_final', 
#                                    word_ngrams = 2,
                                    lr=0.5, 
                                    dim=300, 
                                    epoch=25, 
                                    loss='softmax', 
                                    ws=2, 
                                    pretrained_vectors='model_skipgram.vec'
                                 )
# result_final = model_final.test('test.txt')
# print(result_final.precision)

In [21]:
labels = model_final.predict(X_test, 1)
labels = [int(elem[0]) for elem in labels]
labels_names = le.inverse_transform(labels)

/Users/trokhymovych/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
# First Kaggle commit
a = len(labels_names)
a = [x for x in range(len(labels_names))]
f = {'Id':a, 'y':labels_names}
ff = pd.DataFrame(f)
ff.to_csv('res.csv', index = False)